# Experiment 091: Per-Target Heterogeneous Ensemble

**Rationale**: The strategy-to-get-0-11161 kernel (LB=0.11161) uses DIFFERENT model types for different targets:
- SM target: HistGradientBoostingRegressor (HGB)
- Product 2 & 3: ExtraTreesRegressor (ETR)
- Ensemble: 0.65 * ACS_PCA + 0.35 * Spange

**Why this could change the CV-LB relationship**:
1. Different targets may have different optimal model types
2. SM (starting material) may behave differently than products
3. This is fundamentally different from using the same model for all targets

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[['Residence Time', 'Temperature', 'SOLVENT A NAME', 'SOLVENT B NAME', 'SolventB%']]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[['Residence Time', 'Temperature', 'SOLVENT NAME']]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

In [ ]:
# Load feature lookups
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0).apply(pd.to_numeric, errors='coerce').fillna(0)
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0).apply(pd.to_numeric, errors='coerce').fillna(0)

print(f'ACS PCA: {ACS_PCA_DF.shape}, Spange: {SPANGE_DF.shape}')

In [ ]:
# BetterCatecholModel - exactly as in the kernel
class BetterCatecholModel:
    def __init__(self, feature_table="spange_descriptors", base_type="hgb"):
        self.base_type = base_type
        if feature_table == "acs_pca_descriptors":
            self.lookup = ACS_PCA_DF
        else:
            self.lookup = SPANGE_DF
        self.model = None

    def _vec(self, s):
        return self.lookup.loc[s].values if s in self.lookup.index else np.zeros(self.lookup.shape[1])

    def _build_X(self, X):
        rt = X["Residence Time"].values.reshape(-1, 1)
        temp = X["Temperature"].values.reshape(-1, 1)

        if "SOLVENT NAME" in X.columns:
            S = np.vstack([self._vec(s) for s in X["SOLVENT NAME"]])
            return np.hstack([rt, temp, S])

        frac_b = X["SolventB%"].values.reshape(-1, 1) / 100.0
        A = np.vstack([self._vec(s) for s in X["SOLVENT A NAME"]])
        B = np.vstack([self._vec(s) for s in X["SOLVENT B NAME"]])
        mix = (1 - frac_b) * A + frac_b * B

        return np.hstack([rt, temp, frac_b, mix])

    def train_model(self, X, Y):
        Xf = self._build_X(X)
        y = Y.values

        if self.base_type == "hgb":
            base = HistGradientBoostingRegressor(
                max_depth=7, max_iter=700, learning_rate=0.04
            )
        else:
            base = ExtraTreesRegressor(
                n_estimators=900,
                min_samples_leaf=2,
                random_state=42,
                n_jobs=-1,
            )

        self.model = Pipeline(
            [("scaler", StandardScaler()), ("reg", MultiOutputRegressor(base))]
        )
        self.model.fit(Xf, y)

    def predict(self, X):
        pred = np.clip(self.model.predict(self._build_X(X)), 0, 1)
        return torch.tensor(pred, dtype=torch.double)

print('BetterCatecholModel defined')

In [ ]:
# Per-Target Heterogeneous Ensemble - exactly as in the kernel
class PerTargetEnsembleModel:
    def __init__(self):
        self.targets = ["Product 2", "Product 3", "SM"]
        self.models = {}

        for t in self.targets:
            if t == "SM":
                # HGB for SM target
                self.models[t] = [
                    BetterCatecholModel("acs_pca_descriptors", "hgb"),
                    BetterCatecholModel("spange_descriptors", "hgb"),
                ]
            else:
                # ETR for Product 2 & 3
                self.models[t] = [
                    BetterCatecholModel("acs_pca_descriptors", "etr"),
                    BetterCatecholModel("spange_descriptors", "etr"),
                ]

    def train_model(self, X, Y):
        for t in self.targets:
            y_single = Y[[t]]
            for m in self.models[t]:
                m.train_model(X, y_single)

    def predict(self, X):
        preds = []

        for t in self.targets:
            p1 = self.models[t][0].model.predict(self.models[t][0]._build_X(X))
            p2 = self.models[t][1].model.predict(self.models[t][1]._build_X(X))

            # Ensemble: 0.65 * ACS_PCA + 0.35 * Spange
            pred_t = 0.65 * p1 + 0.35 * p2
            preds.append(pred_t.reshape(-1, 1))

        pred = np.clip(np.hstack(preds), 0, 1)
        return torch.tensor(pred, dtype=torch.double)

print('PerTargetEnsembleModel defined')
print('SM: HGB (gradient boosting)')
print('Product 2 & 3: ETR (random forest variant)')
print('Ensemble: 0.65 * ACS_PCA + 0.35 * Spange')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PerTargetEnsembleModel()  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PerTargetEnsembleModel()  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest previous CV: 0.008092 (CatBoost+XGBoost)')
print(f'Best previous LB: 0.0877 (GP+MLP+LGBM)')
print(f'\nThis (Per-Target Heterogeneous Ensemble): CV {overall_mse:.6f}')

if overall_mse < 0.008092:
    improvement = (0.008092 - overall_mse) / 0.008092 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008092) / 0.008092 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')